# BrkRaw CLI Tutorial

This notebook mirrors the Python API walkthrough using CLI commands.
Commands are run in bash cells so you can see the outputs.

Orientation visualization is omitted here; use the Python notebook for image checks.

## 0. Initialize config and install defaults

BrkRaw stores rules/specs/transforms under a config root (default: `~/.brkraw`).
Install the bundled defaults once per environment so metadata parsing and layout
resolution work for later steps.

Note: Default rules/specs/transforms will continue to evolve and be standardized as the user base grows; the current version is not complete and contributions are welcome.

In [1]:
!brkraw init --install-default --yes

Initialized config at /Users/shlee/.brkraw
Installed transforms: /Users/shlee/.brkraw/transforms/metadata_transforms.py
Installed spec: /Users/shlee/.brkraw/specs/metadata_common.yaml
Installed spec: /Users/shlee/.brkraw/specs/metadata_func.yaml
Installed spec: /Users/shlee/.brkraw/specs/metadata_anat.yaml
Installed transforms: /Users/shlee/.brkraw/transforms/mrs_transforms.py
Installed spec: /Users/shlee/.brkraw/specs/mrs.yaml
Installed rule: /Users/shlee/.brkraw/rules/20-mrs.yaml
Installed rule: /Users/shlee/.brkraw/rules/10-metadata.yaml
Installed pruner spec: /Users/shlee/.brkraw/pruner_specs/deid4share.yaml
Installed 9 default file(s).
Shell helpers already present in /Users/shlee/.zshrc


## 1. Download example data (Git LFS)

This uses the same Git LFS dataset repository as the Python tutorial.

In [2]:
%%bash
set -euo pipefail

git lfs version
mkdir -p data
if [ ! -d data/brkraw-dataset ]; then
  git clone https://github.com/BrkRaw/brkraw-dataset data/brkraw-dataset
fi
git -C data/brkraw-dataset lfs pull

git-lfs/3.7.1 (GitHub; darwin arm64; go 1.25.3)


## 2. Choose a dataset

We will use the same ZIP as the Python tutorial.

In [3]:
%env DATASET_ZIP=data/brkraw-dataset/PV6.0.1/UNC_PV6.0.1_FLASH_TurboRARE_EPI.zip
%env SCAN_ID=3
%env RECO_ID=1

env: DATASET_ZIP=data/brkraw-dataset/PV6.0.1/UNC_PV6.0.1_FLASH_TurboRARE_EPI.zip
env: SCAN_ID=3
env: RECO_ID=1


## 3. Inspect study and scan info

Start with the full summary, then narrow to a specific scan.

In [4]:
%%bash
set -euo pipefail

brkraw info "$DATASET_ZIP"
brkraw info "$DATASET_ZIP" --scope scan -s "$SCAN_ID" --show-reco

[ Study ]
   field     value
Software     Paravision v6.0.1
Opperator    nmrsu
Date         2000-01-01 00:00:00
ID           1
Number       1
InstanceUID  1.2.3.4.5

[ Subject ]
   field     value
ID           phantom
Name         CAMRI
Type         Quadruped
Sex          UNKNOWN
DateOfBirth  2020-01-12 00:00:00
Weight       0.001
Position     Head_Supine

[ Scan(s) ]
 ScanID     Method        Protocol      Dim       DimDesc        NumEchoes   TE (ms)    TR (ms)  FlipAngle (degree)
   3     Bruker:FLASH  1_Localizer_GOP  2    [Spatial, Spatial]  1          [3.368000]  [100]    30                
           Shape     FOV (mm)  NumSlicePack         SliceOrient            ReadOrient     SliceDistance (mm)
         [256, 256]  [30, 30]  3             [axial, sagittal, coronal]  [L_R, H_F, H_F]  [3, 3, 3]         
         SliceGap (mm)  NumAverage  NumRepeat
         [1, 1, 1]      1           1        
 ScanID    Method          Protocol        Dim       DimDesc        NumEchoes  TE (ms) 

[ Scan(s) ]
 ScanID     Method        Protocol      Dim       DimDesc        NumEchoes   TE (ms)    TR (ms)  FlipAngle (degree)
   3     Bruker:FLASH  1_Localizer_GOP  2    [Spatial, Spatial]  1          [3.368000]  [100]    30                
           Shape     FOV (mm)  NumSlicePack         SliceOrient            ReadOrient     SliceDistance (mm)
         [256, 256]  [30, 30]  3             [axial, sagittal, coronal]  [L_R, H_F, H_F]  [3, 3, 3]         
         SliceGap (mm)  NumAverage  NumRepeat
         [1, 1, 1]      1           1        
 [ Reco(s) ]
 RecoID  value
   1     MAGNITUDE_IMAGE


## 4. Search parameters

Search for a parameter key in scan-level parameter files.

In [5]:
%%bash
set -euo pipefail

brkraw params "$DATASET_ZIP" -k PVM_RepetitionTime -s "$SCAN_ID"

method:
  PVM_RepetitionTime: 100



## 5. Convert to NIfTI

Convert the selected scan and write outputs to a local folder.

In [6]:
%%bash
set -euo pipefail

mkdir -p outputs-cli
brkraw convert "$DATASET_ZIP" -s "$SCAN_ID" -r "$RECO_ID" -o outputs-cli
ls -1 outputs-cli | head -n 5

Wrote NIfTI: outputs-cli/sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack1.nii.gz
Wrote NIfTI: outputs-cli/sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack2.nii.gz
Wrote NIfTI: outputs-cli/sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack3.nii.gz


sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack1.nii.gz
sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack2.nii.gz
sub-phantom_study-1_scan-3_1_Localizer_GOP_slpack3.nii.gz


## 6. Optional: use session defaults

Session defaults let you avoid repeating path/scan/reco across commands.
They only apply within the shell used by this cell.

In [7]:
%%bash
DATASET_ZIP=./data/brkraw-dataset/PV6.0.1/UNC_PV6.0.1_FLASH_TurboRARE_EPI.zip
SCAN_ID=3
RECO_ID=1

eval "$(brkraw session set -p $DATASET_ZIP -s $SCAN_ID -r $RECO_ID)"
brkraw info --scope scan
brkraw params -k VisuAcqEchoTime -f visu_pars

[ Scan(s) ]
 ScanID     Method        Protocol      Dim       DimDesc        NumEchoes   TE (ms)    TR (ms)  FlipAngle (degree)
   3     Bruker:FLASH  1_Localizer_GOP  2    [Spatial, Spatial]  1          [3.368000]  [100]    30                
           Shape     FOV (mm)  NumSlicePack         SliceOrient            ReadOrient     SliceDistance (mm)
         [256, 256]  [30, 30]  3             [axial, sagittal, coronal]  [L_R, H_F, H_F]  [3, 3, 3]         
         SliceGap (mm)  NumAverage  NumRepeat
         [1, 1, 1]      1           1        


visu_pars:
  VisuAcqEchoTime:
  - 3.368

